# Compressible Gas Dynamics

In [1]:
# Add path to src/CARPy, in case notebook is running locally
import os, sys, warnings
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..\\..\\..\\src")))
warnings.simplefilter("ignore")  # <-- Suppress warnings

***
## Introduction

Frequently encountered in aeronautical engineering is the need to account for compressible gas effects.
CARPy supports designers by providing several gas dynamic calculators, including methods for:

1. `IsentropicFlow`: when changes in flow are small and gradual.
2. `NormalShock` (relations): when shocks form with wave fronts normal to the freestream flow.
3. `ObliqueShock` (relations): when shocks form with wave fronts are inclined with respect to the freestream flow.
4. `ExpansionFan`: when supersonic flow encounters convex geometry.
5. `RayleighFlow`: describing frictionless, non-adiabatic flow through a constant area duct where effects of heat addition or rejection are considered.
6. `FannoFlow`: describing adiabatic, non-frictionless flow through a constant area duct.

CARPy does not yet include unit tests to validate the outputs of any compressible gas dynamics methods.

***
## 1) `IsentropicFlow` methods

Start with relevant imports

In [2]:
from carpy.gaskinetics import IsentropicFlow as IF

publicmethods = [x for x in dir(IF) if not x.startswith("__")]
print(publicmethods)

['A_Astar', 'M', 'T_Tstar', 'T_Tt', 'mdot', 'mu', 'nu', 'p_pstar', 'p_pt', 'rho_rhostar', 'rho_rhot']


Any method written as `X_X0` indicates that the method will return the ratio of static to stagnation quantities (e.g. $T / T_0$):

In [3]:
# Static to stagnation temperature ratio at Mach 2
print("T/Tt @ Mach2 =", IF.T_Tt(M=2))

T/Tt @ Mach2 = [0.55555556]


Also available to users are static to sonic quantity ratios (e.g. $T/T^*$):

In [4]:
# Static to stagnation temperature ratio at Mach 2, gamma=1.2
print("T/Tt @ Mach2 =", IF.T_Tstar(M=2, gamma=1.2))

T/Tt @ Mach2 = [0.78571429]


***
## 2) `NormalShock` methods

Start with relevant imports

In [5]:
from carpy.gaskinetics import NormalShock as NS

publicmethods = [x for x in dir(NS) if not x.startswith("__")]
print(publicmethods)

['M2', 'T02_T01', 'T2_T1', 'V2_V1', 'p02_p01', 'p2_p1', 'rho2_rho1']


Given upstream conditions, it is trivial for users to evaluate conditions downstream of a normal shock:

In [6]:
# Set upstream conditions
M1 = 2
gamma = 1.3
conditions = {"M1":M1, "gamma": gamma}

print(f"M1 (upstream)...... {M1:.1f}")
print(f"gamma (upstream)... {gamma:.1f}")
print("-"*20)
print(f"M2 ................ {NS.M2(**conditions)}")
print(f"p2/p1 ............. {NS.p2_p1(**conditions)}")
print(f"T2/T1 ............. {NS.T2_T1(**conditions)}")

M1 (upstream)...... 2.0
gamma (upstream)... 1.3
--------------------
M2 ................ [0.56287804]
p2/p1 ............. [4.39130435]
T2/T1 ............. [1.52741021]


***
## 3) `ObliqueShock` methods

Start with relevant imports

In [7]:
import numpy as np

from carpy.gaskinetics import ObliqueShock as OS

publicmethods = [x for x in dir(OS) if not x.startswith("__")]
print(publicmethods)

['M2', 'T02_T01', 'T2_T1', 'beta', 'p02_p01', 'p2_p1', 'rho2_rho1', 'theta']


`ObliqueShock` methods are very similar to those contained in `NormalShock`, with the exception that a flow deflection parameter `theta` is introduced:

In [8]:
# Set upstream conditions
M1 = 2
gamma = 1.3
theta = np.radians(6)
conditions = {"M1":M1, "gamma": gamma, "theta":theta}

print(f"M1 (upstream)...... {M1:.1f}")
print(f"gamma (upstream)... {gamma:.1f}")
print(f"theta (upstream)... {np.degrees(theta):.3f} [deg]")
print("-"*20)
print(f"M2 ................ {OS.M2(**conditions)}")
print(f"p02/p01 ........... {OS.p02_p01(**conditions)}")
print(f"T2/T1 ............. {OS.T2_T1(**conditions)}")

M1 (upstream)...... 2.0
gamma (upstream)... 1.3
theta (upstream)... 6.000 [deg]
--------------------
M2 ................ (array([1.80781852]), array([0.57089155]))
p02/p01 ........... (array([0.99679791]), array([0.7022005]))
T2/T1 ............. (array([1.07365893]), array([1.52542564]))


Help!
Why does the calculator give me two solutions for one set of upstream conditions?
The answer is quite simply that the `ObliqueShock` calculator provides both weak and strong shock solutions, respectively (note how M2 has a supersonic solution with a weak shock, and subsonic solution in strong shock cases).

Notice that the downstream stagnation pressure $p_{02}$ is much closer to that of $p_{01}$ in a weak shock case; meaning that well-designed supersonic engine inlets recover pressure far better through weak shocks than strong shocks - and with a lesser temperature penalty!

For all flows exists a maximum flow deflection angle, beyond which the oblique shock detaches from the corner and is replaced with a detached bow shock.
This is apparent when the shock angle solutions return 'not a number' answers.

In [9]:
OS.beta(M1=M1, gamma=gamma, theta=np.radians(25))

(array([nan]), array([nan]))

***
## 4) `ExpansionFan` objects

Start with relevant imports

In [10]:
from carpy.gaskinetics import ExpansionFan as EF

publicmethods = [x for x in dir(EF) if not x.startswith("__")]
print(publicmethods)

['M1', 'M2', 'T2_T1', 'gamma', 'p2_p1', 'rho2_rho1', 'theta']


`ExpansionFan` methods are unique in that unlike other classes in this module, many attributes of `ExpansionFan` depend on computationally costly root-finding methods.
For this reason, `ExpansionFan` is made an instantiable object - encouraging efficient use of broadcasted inputs that are then cached to eliminate recompute time.

The alternative to instantiating would've been to use class methods like the other classes, however, it was far more convenient to use persistent fan objects.

In [11]:
# Set upstream conditions
M1 = [2, 3, 3]
gamma = [1.3, 1.3, 1.2]
theta = np.radians(6)
conditions = {"M1":M1, "gamma": gamma, "theta":theta}

# Compute downstream (fan) conditions
fan = EF(**conditions)

print(f"M1 (upstream)...... {M1}")
print(f"gamma (upstream)... {gamma}")
print(f"theta (upstream)... {np.degrees(theta):.1f} [deg]")
print("-"*20)
print(f"M2 ................ {fan.M2}")
print(f"p2/p1 ............. {fan.p2_p1}")
print(f"T2/T1 ............. {fan.T2_T1}")

M1 (upstream)...... [2, 3, 3]
gamma (upstream)... [1.3, 1.3, 1.2]
theta (upstream)... 6.0 [deg]
--------------------
M2 ................ [2.19788692 3.27345585 3.21746633]
p2/p1 ............. [0.72254298 0.63745115 0.6620146 ]
T2/T1 ............. [0.92774812 0.90130621 0.93356507]


Notice how the results have been broadcasted to deal with only one `theta` value.

***
## 5) `RayleighFlow` methods

Start with relevant imports

In [12]:
from carpy.gaskinetics import RayleighFlow as RF

publicmethods = [x for x in dir(RF) if not x.startswith("__")]
print(publicmethods)

['DeltaS', 'H', 'T0_T0star', 'T_Tstar', 'V_Vstar', 'p0_p0star', 'p_pstar', 'rho_rhostar']


In [13]:
# Set upstream conditions
M = 2.0
gamma = 1.3
conditions = {"M":M, "gamma": gamma}

print(f"M (upstream)....... {M:.1f}")
print(f"gamma (upstream)... {gamma:.1f}")
print("-"*20)
print(f"DeltaS ............ {RF.DeltaS(**conditions)}")
print(f"p/p* .............. {RF.p_pstar(**conditions)}")
print(f"T/T* .............. {RF.T_Tstar(**conditions)}")

M (upstream)....... 2.0
gamma (upstream)... 1.3
--------------------
DeltaS ............ [-0.36814594]
p/p* .............. [0.37096774]
T/T* .............. [0.55046826]


***
## 6) `FannoFlow` methods

Start with relevant imports

In [14]:
from carpy.gaskinetics import FannoFlow as FF

publicmethods = [x for x in dir(FF) if not x.startswith("__")]
print(publicmethods)

['DeltaS', 'H', 'T0_T0star', 'T_Tstar', 'V_Vstar', 'p0_p0star', 'p_pstar', 'rho0_rho0star', 'rho_rhostar']


In [15]:
# Set upstream conditions
M = 2
gamma = 1.3
conditions = {"M":M, "gamma": gamma}

print(f"M (upstream)....... {M:.1f}")
print(f"gamma (upstream)... {gamma:.1f}")
print("-"*20)
print(f"DeltaS ............ {FF.DeltaS(**conditions)}")
print(f"p/p* .............. {FF.p_pstar(**conditions)}")
print(f"T/T* .............. {FF.T_Tstar(**conditions)}")

M (upstream)....... 2.0
gamma (upstream)... 1.3
--------------------
DeltaS ............ [-0.13217984]
p/p* .............. [0.42389562]
T/T* .............. [0.71875]
